In [73]:
# change directory
import os
os.chdir('C:/Users/theon/OneDrive/Desktop/Giga Projects/seinfeld text gen')

In [74]:
# basically a function to break down sequences of words into the markov dictionaries (e.g. ("the", "quick", "brown") : ["Fox"])
def add_to_model(model, n, seq):            # function for creating n-grams from given sequence; model=n-gram storage bin; n=length of n-grams; seq=thing to create n-grams from
    seq = list(seq[:]) + [None]             # create copy of seq WITHOUT overwriting original (hence, list(seq[:]); none is to signify the end of the sequence for model purposes
    for i in range(len(seq)-n):             # iterate through seq to the point where there are enough words remaining to form n-gram (hence -n)
        gram = tuple(seq[i:i+n])            # create tuple of n consecutive words, starting at position i; use tuples bc of their immutability
        next_item = seq[i+n]                # store the next item that appears after the end of the previous n-gram
        if gram not in model:               # check if we have already stored this certain n-gram
            model[gram] = []                # create dictionary entry for certain gram
        model[gram].append(next_item)       # append another "outcome" of the n-gram to the n-gram's dictionary entry

# basically a function to use the add_to_model to actually have output
def markov_model(n, seq):                   # function for 
    model = {}                              # here, we create an empty dictionary into which we are going to put the n-grams as a string or list.
    add_to_model(model, n, seq)             #we call the add_to_model function on the sequence.
    return model

In [75]:
import random

def gen_from_model(n, model, start=None, max_gen=100):              # function for generating text
    if start is None:                                               # if we do not specify place to start, start from random sequence
        start = random.choice(list(model.keys()))                   # the keys ("".keys") are the sequences in the model ("model") we created earlier.
    output = list(start)                                            # create first element of output
    for i in range(max_gen):                                        # iterate through number of generated words we want
        start = tuple(output[-n:])                                  # we format that ngram ("output[-n:]") as a tuple; output[-n:] grabs the last n items from the output
        next_item = random.choice(model[start])                     # check the dictionary entry corresponding to the "start" n-gram, and choose a random follow-up word
        if next_item is None:                                       
            break                                                   # exit the for-loop
        else:
            output.append(next_item)                                # grab the next n-gram to continue the process
    return output

In [76]:
# set n-grams and starting point
n_grams = 3
starting_pos = None

# generate markov chain
sein_word_model = markov_model(n_grams, open("standardized_seinfeld_script.txt").read().split())
generated_words = gen_from_model(n_grams, sein_word_model, starting_pos, max_gen=1000)

In [77]:
# change dir again
os.chdir('C:/Users/theon/OneDrive/Desktop/Giga Projects/seinfeld text gen/sein_markov')

In [78]:
# write generated text to file
with open("generated_sein.txt", "w", encoding="utf-8") as file:
    for word in generated_words:
        if any(char in word for char in [':', '[', '(']):
            word = "\n " + word
        file.write(word+' ')

with open("generated_sein.txt", "r") as file:
    lines = file.readlines()
    lines[0] = f"GENERATED SEINFELD SCRIPT: {n_grams}-grams starting from {starting_pos} \n"

with open("generated_sein.txt", "w", encoding="utf-8") as file:
    file.writelines(lines)